In [281]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from typing import List

In [282]:
caminho_base = '../data/processed/df_processed_v2.parquet'

In [283]:
base_completa = pd.read_parquet(caminho_base)

In [284]:
base_completa

,NOME_AEROPORTO_ORIGEM,CIDADE_AEROPORTO_ORIGEM,UF_AEROPORTO_ORIGEM,SIGLA_ICAO_EMPRESA_AEREA,NUMERO_VOO,CODIGO_TIPO_LINHA,SIGLA_ICAO_AEROPORTO_ORIGEM,SIGLA_ICAO_AEROPORTO_DESTINO,PARTIDA_PREVISTA,PARTIDA_REAL,...,INDICE_SEMANA_CHEGADA_REAL,INDICE_SEMANA_CHEGADA_PREVISTA,FAIXA_ATRASO_PARTIDA,FAIXA_ATRASO_CHEGADA,FLAG_ATRASO_PARTIDA,FLAG_ATRASO_CHEGADA,FLAG_FIM_SEMANA_PARTIDA_PREVISTA,FLAG_FIM_SEMANA_PARTIDA_REAL,FLAG_FIM_SEMANA_CHEGADA_PREVISTA,FLAG_FIM_SEMANA_CHEGADA_REAL
0,BAURU/AREALVA,AREALVA,SP,AZU,2455,N,SBAE,SBPS,2022-01-02 00:05:00,2022-01-01 23:35:00,...,6,6,No horário,No horário,Vôo Atrasado,Vôo Atrasado,Fim de Semana,Fim de Semana,Fim de Semana,Fim de Semana
1,BAURU/AREALVA,AREALVA,SP,AZU,2455,N,SBAE,SBPS,2022-01-09 01:00:00,2022-01-09 00:48:00,...,6,6,No horário,No horário,Vôo Atrasado,Vôo Atrasado,Fim de Semana,Fim de Semana,Fim de Semana,Fim de Semana
2,BAURU/AREALVA,AREALVA,SP,AZU,2455,N,SBAE,SBPS,2022-01-23 11:05:00,2022-01-23 10:54:00,...,6,6,No horário,No horário,Vôo Atrasado,Vôo Atrasado,Fim de Semana,Fim de Semana,Fim de Semana,Fim de Semana
3,BAURU/AREALVA,AREALVA,SP,AZU,2455,N,SBAE,SBPS,2022-01-30 11:05:00,2022-01-30 11:01:00,...,6,6,No horário,No horário,Vôo Atrasado,Vôo Atrasado,Fim de Semana,Fim de Semana,Fim de Semana,Fim de Semana
4,BAURU/AREALVA,AREALVA,SP,AZU,2455,N,SBAE,SBPS,2022-02-26 23:00:00,2022-02-26 22:56:00,...,6,6,No horário,No horário,Vôo Atrasado,Vôo Atrasado,Fim de Semana,Fim de Semana,Fim de Semana,Fim de Semana
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717352,DEPUTADO LUÍS EDUARDO MAGALHÃES,SALVADOR,BA,ABJ,1416,N,SBSV,SIRI,1900-01-01 00:00:00,2022-12-15 14:34:00,...,3,0,Atraso de mais de 4 horas,Atraso de mais de 4 horas,Vôo no horário,Vôo no horário,Dia Útil,Dia Útil,Dia Útil,Dia Útil
717353,DEPUTADO LUÍS EDUARDO MAGALHÃES,SALVADOR,BA,ABJ,1416,N,SBSV,SIRI,2022-12-18 15:35:00,2022-12-18 15:41:00,...,6,6,Atraso de até 30 minutos,No horário,Vôo no horário,Vôo Atrasado,Fim de Semana,Fim de Semana,Fim de Semana,Fim de Semana
717354,DEPUTADO LUÍS EDUARDO MAGALHÃES,SALVADOR,BA,ABJ,1416,N,SBSV,SIRI,1900-01-01 00:00:00,2022-12-22 14:46:00,...,3,0,Atraso de mais de 4 horas,Atraso de mais de 4 horas,Vôo no horário,Vôo no horário,Dia Útil,Dia Útil,Dia Útil,Dia Útil
717355,DEPUTADO LUÍS EDUARDO MAGALHÃES,SALVADOR,BA,ABJ,1416,N,SBSV,SIRI,2022-12-25 15:35:00,2022-12-25 15:11:00,...,6,6,No horário,No horário,Vôo Atrasado,Vôo Atrasado,Fim de Semana,Fim de Semana,Fim de Semana,Fim de Semana


- Situação Vôo

In [285]:
def obter_situacao_voo(
        sigla_aeroporto: str,
        mes_partida_prevista: List[int],
        codigo_tipo_linha: str, 
        sigla_empresa: str = None,
       
):
    query = (
            f' SIGLA_ICAO_AEROPORTO_ORIGEM == "{sigla_aeroporto}" '
            f' and MES_PARTIDA_PREVISTA in {mes_partida_prevista}'  # TAM SBRP 1
            f' and CODIGO_TIPO_LINHA == "{codigo_tipo_linha}"'
    )
    colunas = [
            'SITUACAO_VOO', 
            'MES_PARTIDA_PREVISTA', 
            'NOME_MES_PARTIDA_PREVISTA', 
            'SIGLA_ICAO_AEROPORTO_ORIGEM',
            'CODIGO_TIPO_LINHA'
    ]

    if sigla_empresa is not None:
        query += f' and SIGLA_ICAO_EMPRESA_AEREA == "{sigla_empresa}" '
        colunas.append('SIGLA_ICAO_EMPRESA_AEREA')
   
    dataframe = pd.read_parquet(caminho_base, columns=colunas)
    total = dataframe.query(query)[colunas].value_counts()
    percentual = dataframe.query(query)[colunas].value_counts( normalize=True,) * 100
    dataframe = pd.concat([total, percentual], axis=1).reset_index()
    dataframe['TOTAL_SITUACAO'] = dataframe['count'].astype('int32')
    dataframe['PROPORCAO'] = dataframe['proportion'].astype('float32')
    dataframe = dataframe.sort_values(by=['SITUACAO_VOO', 'MES_PARTIDA_PREVISTA']) 
    dataframe.drop(['proportion', 'count', 'CODIGO_TIPO_LINHA'], axis=1, inplace=True,)
    dataframe['SITUACAO_VOO'] =  dataframe['SITUACAO_VOO'].astype('string')
    if sigla_empresa is not None:
        dataframe = dataframe.sort_values(by='MES_PARTIDA_PREVISTA')
    return dataframe 

In [286]:
dataframe = obter_situacao_voo(
    sigla_aeroporto='SBRP', 
    mes_partida_prevista=[i for i in range(1, 13)], # [1,2]
    codigo_tipo_linha='N',
    sigla_empresa='TAM'
)
dataframe

,SITUACAO_VOO,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,SIGLA_ICAO_AEROPORTO_ORIGEM,SIGLA_ICAO_EMPRESA_AEREA,TOTAL_SITUACAO,PROPORCAO
2,REALIZADO,1,janeiro,SBRP,TAM,41,11.781610
10,REALIZADO,2,fevereiro,SBRP,TAM,20,5.747127
4,REALIZADO,3,março,SBRP,TAM,23,6.609195
8,REALIZADO,4,abril,SBRP,TAM,21,6.034483
11,REALIZADO,5,maio,SBRP,TAM,20,5.747127
7,REALIZADO,6,junho,SBRP,TAM,22,6.321839
9,REALIZADO,7,julho,SBRP,TAM,21,6.034483
5,REALIZADO,8,agosto,SBRP,TAM,23,6.609195
6,REALIZADO,9,setembro,SBRP,TAM,23,6.609195
3,REALIZADO,10,outubro,SBRP,TAM,24,6.896552


In [287]:
dataframe['SITUACAO_VOO'].unique()

<StringArray>
['REALIZADO', 'CANCELADO']
Length: 2, dtype: string

In [288]:
fig = px.bar(
    dataframe,
    x='NOME_MES_PARTIDA_PREVISTA',
    y='TOTAL_SITUACAO',
    color='SITUACAO_VOO',
    barmode='group',
    text_auto='0'
)

fig.show()

- Variação  da Situação do Voo em relação ao mês anterior (Não expressar em porcentagem)

In [289]:
def obter_variacao_situacao_voo(sigla_aeroporto: str, mes_partida_prevista: List[int], codigo_tipo_linha: str, situacao_voo: str,  sigla_empresa: str = None):
    dataframe = obter_situacao_voo(
        sigla_aeroporto=sigla_aeroporto, 
        mes_partida_prevista=mes_partida_prevista, # [1,2]
        codigo_tipo_linha=codigo_tipo_linha,
        sigla_empresa=sigla_empresa
    )
    dataframe.drop('PROPORCAO', axis=1, inplace=True)
    dataframe['TOTAL_SITUACAO_MES_ANTERIOR'] = dataframe.groupby('SITUACAO_VOO')['TOTAL_SITUACAO'].shift(1)
    dataframe.fillna(0,axis=1,inplace=True)
    dataframe = dataframe.query(f' SITUACAO_VOO == "{situacao_voo}" ')
    return dataframe

In [290]:
dataframe = obter_variacao_situacao_voo(
    sigla_aeroporto='SBRP', 
    mes_partida_prevista=[i for i in range(1, 13)], # [1,2]
    codigo_tipo_linha='N',
    situacao_voo='REALIZADO'
)
dataframe 

,SITUACAO_VOO,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,SIGLA_ICAO_AEROPORTO_ORIGEM,TOTAL_SITUACAO,TOTAL_SITUACAO_MES_ANTERIOR
0,REALIZADO,1,janeiro,SBRP,582,0
11,REALIZADO,2,fevereiro,SBRP,249,582.0
9,REALIZADO,3,março,SBRP,345,249.0
5,REALIZADO,4,abril,SBRP,381,345.0
1,REALIZADO,5,maio,SBRP,433,381.0
3,REALIZADO,6,junho,SBRP,387,433.0
10,REALIZADO,7,julho,SBRP,250,387.0
2,REALIZADO,8,agosto,SBRP,409,250.0
7,REALIZADO,9,setembro,SBRP,368,409.0
6,REALIZADO,10,outubro,SBRP,375,368.0


In [291]:
fig = go.Figure()


fig.add_trace(
    go.Bar(
        x=dataframe['NOME_MES_PARTIDA_PREVISTA'],
        y=dataframe['TOTAL_SITUACAO'],
        name='Mês atual',

    )
)


fig.add_trace(
    go.Scatter(
        x=dataframe['NOME_MES_PARTIDA_PREVISTA'],
        y=dataframe['TOTAL_SITUACAO_MES_ANTERIOR'],
        name='Mês anterior'
    )
)
fig.update_layout(
    title='Teste'
)
fig.show()

- Analise da variação percentual da SITUAÇÂO voo Filtrando por Aerporto origem por mês compare com o mês anterior (Talvez com vísual cartões) deixe filtro facultativo por empresa)

In [292]:
def calcular_variacao(linha):
    try:
        total_atual = linha['TOTAL_SITUACAO']
        total_mes_anterior = linha['TOTAL_SITUACAO_MES_ANTERIOR']
        variacao = ((total_atual - total_mes_anterior) / total_mes_anterior) * 100
    except ZeroDivisionError:
        variacao = 0
    return round(variacao, 2)

In [293]:
def obter_variacao_percentual(sigla_aeroporto: str, situacao_voo: str, mes_partida: int, codigo_tipo_linha: str,  sigla_empresa: str = None):
    colunas = [
        'SITUACAO_VOO',
        'MES_PARTIDA_PREVISTA',
        'NOME_MES_PARTIDA_PREVISTA',
        'SIGLA_ICAO_AEROPORTO_ORIGEM',
        'NOME_AEROPORTO_ORIGEM',
        'NOME_EMPRESA',
        'CODIGO_TIPO_LINHA'
    ]
    query = f' SIGLA_ICAO_AEROPORTO_ORIGEM == "{sigla_aeroporto}" and  CODIGO_TIPO_LINHA == "{codigo_tipo_linha}" '
    if sigla_empresa is not None:
        query += f' and SIGLA_ICAO_EMPRESA_AEREA == "{sigla_empresa}"  '
        colunas.append('SIGLA_ICAO_EMPRESA_AEREA')

    dataframe = pd.read_parquet(caminho_base, columns=colunas)
    dataframe = dataframe.query(query)
    dataframe[['SITUACAO_VOO','NOME_MES_PARTIDA_PREVISTA' ]] = dataframe[['SITUACAO_VOO','NOME_MES_PARTIDA_PREVISTA' ]].astype('string')
    dataframe = dataframe.groupby(['SITUACAO_VOO', 'NOME_MES_PARTIDA_PREVISTA', 'MES_PARTIDA_PREVISTA']).agg(
        TOTAL_SITUACAO=('SITUACAO_VOO', 'count')
    ).sort_values(by=['SITUACAO_VOO','MES_PARTIDA_PREVISTA']).reset_index()
    dataframe['TOTAL_SITUACAO_MES_ANTERIOR'] = dataframe.groupby('SITUACAO_VOO')['TOTAL_SITUACAO'].shift(1)
    dataframe.fillna(0, axis=1, inplace=True)
    dataframe['PERCENTUAL_VARIACAO'] = dataframe.apply(calcular_variacao, axis=1)
    query = f'SITUACAO_VOO == "{situacao_voo}" and MES_PARTIDA_PREVISTA == {mes_partida}'

    dataframe = dataframe.query(query)[['TOTAL_SITUACAO', 'TOTAL_SITUACAO_MES_ANTERIOR', 'PERCENTUAL_VARIACAO']]

    return dataframe

In [294]:
dataframe = obter_variacao_percentual(
    sigla_aeroporto='SBRP',
    sigla_empresa='TAM',
    mes_partida=2, 
    situacao_voo='REALIZADO',
    codigo_tipo_linha='N'
)
dataframe

,TOTAL_SITUACAO,TOTAL_SITUACAO_MES_ANTERIOR,PERCENTUAL_VARIACAO
3,20,41.0,-51.22


In [295]:
fig = go.Figure(
    go.Indicator(
        mode='number+delta',
        value=dataframe['TOTAL_SITUACAO'].values[0],
        delta={
            'position': 'top', 
            'reference': dataframe['TOTAL_SITUACAO_MES_ANTERIOR'].values[0], 
            'relative': True,
            'valueformat': '.2%'
        },
        title={
            'text': 'Total Situação Vôos Realizados'
        },
    )
)
fig.show()

- Compare total de vôos realizado no fim de semana em relação por dia útil por mês

In [296]:
def obter_dados_voos_dia_semana(sigla_aeroporto: str, mes_partida: int, sigla_empresa: str = None):
    colunas = [
        'SITUACAO_VOO',
        'MES_PARTIDA_PREVISTA',
        'NOME_MES_PARTIDA_PREVISTA',
        'SIGLA_ICAO_AEROPORTO_ORIGEM',
        'NOME_AEROPORTO_ORIGEM',
        'NOME_EMPRESA',
        'INDICE_SEMANA_PARTIDA_PREVISTA',
        'DIA_DA_SEMANA_PARTIDA_PREVISTA'
    ]
    query = f' SIGLA_ICAO_AEROPORTO_ORIGEM == "{sigla_aeroporto}" and MES_PARTIDA_PREVISTA == {mes_partida} '
    if sigla_empresa is not None:
        colunas.append('SIGLA_ICAO_EMPRESA_AEREA')
        query += f'SIGLA_ICAO_EMPRESA_AEREA == "{sigla_empresa}" '

    dataframe = pd.read_parquet(caminho_base, columns=colunas)
    dataframe = dataframe.query(query)
    dataframe[['DIA_DA_SEMANA_PARTIDA_PREVISTA','SITUACAO_VOO']] = dataframe[['DIA_DA_SEMANA_PARTIDA_PREVISTA','SITUACAO_VOO']].astype('string')
    dataframe = dataframe.groupby(['DIA_DA_SEMANA_PARTIDA_PREVISTA', 'INDICE_SEMANA_PARTIDA_PREVISTA', 'SITUACAO_VOO']).agg(
        TOTAL_VOOS=('DIA_DA_SEMANA_PARTIDA_PREVISTA', 'count')
    ).reset_index()
    dataframe = dataframe.sort_values(by=['SITUACAO_VOO','INDICE_SEMANA_PARTIDA_PREVISTA'])
    return dataframe

In [297]:
dataframe = obter_dados_voos_dia_semana(
    sigla_aeroporto='SBRP',
    mes_partida=1
)
dataframe

,DIA_DA_SEMANA_PARTIDA_PREVISTA,INDICE_SEMANA_PARTIDA_PREVISTA,SITUACAO_VOO,TOTAL_VOOS
6,Segunda-feira,0,CANCELADO,3
12,Terça-feira,1,CANCELADO,1
2,Quarta-feira,2,CANCELADO,2
4,Quinta-feira,3,CANCELADO,5
8,Sexta-feira,4,CANCELADO,4
10,Sábado,5,CANCELADO,1
0,Domingo,6,CANCELADO,5
7,Segunda-feira,0,REALIZADO,329
13,Terça-feira,1,REALIZADO,41
3,Quarta-feira,2,REALIZADO,42


In [298]:
fig = px.bar(
    dataframe, 
    x='DIA_DA_SEMANA_PARTIDA_PREVISTA', 
    y='TOTAL_VOOS', 
    text_auto='0',
    color='SITUACAO_VOO',
    barmode='group'
)
fig.update_traces(
    textposition='outside'
)
fig.show()

<p> O volume de atraso de saída no fim de semana, é maior ou menor em relação aos outros dias uteis</p>

In [299]:
def obter_variacao_incidencia(
        mes: int, 
        sigla_aeroporto: str, 
        path_coluna_analise: str,
        sigla_empresa_aerea: str = None,
    ) -> pd.DataFrame:
    if sigla_empresa_aerea is None:
        query = f'SIGLA_ICAO_AEROPORTO_ORIGEM == "{sigla_aeroporto}"  and MES_{path_coluna_analise}_PREVISTA == {mes}'
    else:
        query = f' SIGLA_ICAO_AEROPORTO_ORIGEM == "{sigla_aeroporto}" ' \
                f' and SIGLA_ICAO_EMPRESA_AEREA == "{sigla_empresa_aerea}" '\
                f' and MES_{path_coluna_analise}_PREVISTA == {mes} '
    colunas = [
        'NOME_AEROPORTO_ORIGEM',
        'SIGLA_ICAO_EMPRESA_AEREA',
        'SIGLA_ICAO_AEROPORTO_ORIGEM',
        'NOME_EMPRESA',
        f'MES_{path_coluna_analise}_PREVISTA',
        f'NOME_MES_{path_coluna_analise}_PREVISTA',
        f'DIA_DA_SEMANA_{path_coluna_analise}_PREVISTA',
        f'INDICE_SEMANA_{path_coluna_analise}_PREVISTA',
        f'FAIXA_ATRASO_{path_coluna_analise}',
        f'FLAG_ATRASO_{path_coluna_analise}',
        f'FLAG_FIM_SEMANA_{path_coluna_analise}_PREVISTA'
    ]
    dataframe = pd.read_parquet(caminho_base, columns=colunas)
    dataframe = dataframe.query(query)
    dataframe[
        [
            f'NOME_MES_{path_coluna_analise}_PREVISTA', 
            f'DIA_DA_SEMANA_{path_coluna_analise}_PREVISTA', 
            f'FAIXA_ATRASO_{path_coluna_analise}'
        ]
    ] = dataframe[
        [
            f'NOME_MES_{path_coluna_analise}_PREVISTA', 
            f'DIA_DA_SEMANA_{path_coluna_analise}_PREVISTA', 
            f'FAIXA_ATRASO_{path_coluna_analise}'
        ]
    ].astype('string')
    dataframe = dataframe.groupby(
        [
            f'FLAG_ATRASO_{path_coluna_analise}', 
            f'FLAG_FIM_SEMANA_{path_coluna_analise}_PREVISTA'
        ]
    ).agg(
        TOTAL_INCIDENCIA=(f'FLAG_ATRASO_{path_coluna_analise}', 'count')
    ).reset_index()
    dataframe[f'TOTAL_INCIDENCIA_GERAL_{path_coluna_analise}'] = dataframe.groupby(f'FLAG_ATRASO_{path_coluna_analise}')['TOTAL_INCIDENCIA'].transform('sum')
    dataframe[f'PERCENTUAL_INCIDENCIA_{path_coluna_analise}'] = round((dataframe['TOTAL_INCIDENCIA'] * 100) / dataframe[f'TOTAL_INCIDENCIA_GERAL_{path_coluna_analise}'], 2)
    return dataframe

In [300]:
obter_variacao_incidencia(mes=1, sigla_aeroporto='SBRP', path_coluna_analise='PARTIDA')

,FLAG_ATRASO_PARTIDA,FLAG_FIM_SEMANA_PARTIDA_PREVISTA,TOTAL_INCIDENCIA,TOTAL_INCIDENCIA_GERAL_PARTIDA,PERCENTUAL_INCIDENCIA_PARTIDA
0,Vôo Atrasado,Dia Útil,160,227,70.48
1,Vôo Atrasado,Fim de Semana,67,227,29.52
2,Vôo no horário,Dia Útil,346,378,91.53
3,Vôo no horário,Fim de Semana,32,378,8.47


- Incidencia de atraso de Partida / Chegada total por aeroporto (Análise por Flag)

In [301]:
def obter_variacao_flag_atraso(
        sigla_aeroporto: str,
        flag_atraso: str,
        flag_partida_chegada: str,
        sigla_empresa: str = None,
        
    ):

    colunas = [
        'NOME_AEROPORTO_ORIGEM',
        'SIGLA_ICAO_AEROPORTO_ORIGEM',
        'NOME_EMPRESA',
        f'MES_{flag_partida_chegada}_PREVISTA',
        f'NOME_MES_{flag_partida_chegada}_PREVISTA',
        f'DIA_DA_SEMANA_{flag_partida_chegada}_PREVISTA',
        'INDICE_SEMANA_PARTIDA_PREVISTA',
        f'FLAG_ATRASO_{flag_partida_chegada}',
        'FLAG_FIM_SEMANA_PARTIDA_PREVISTA'
    ]
    query = (
        f'SIGLA_ICAO_AEROPORTO_ORIGEM == "{sigla_aeroporto}" ' 
        f' and FLAG_ATRASO_{flag_partida_chegada} == "{flag_atraso}" '
    )
  
    if sigla_empresa is not None:
        query += f' SIGLA_ICAO_EMPRESA_AEREA == "{sigla_empresa}"'
        colunas.append('SIGLA_ICAO_EMPRESA_AEREA')

    dataframe = pd.read_parquet(r'E:\projetos\nova_pasta\dashboard_dados_voos_python\data\processed\df_processed_v2.parquet', columns=colunas)
    dataframe = dataframe.query(query)
   
    dataframe[f'NOME_MES_{flag_partida_chegada}_PREVISTA'] = dataframe[f'NOME_MES_{flag_partida_chegada}_PREVISTA'].astype('string')
    
    dataframe = dataframe.groupby(
        [
            f'FLAG_ATRASO_{flag_partida_chegada}', 
            f'NOME_MES_{flag_partida_chegada}_PREVISTA', 
            f'MES_{flag_partida_chegada}_PREVISTA'
        ]
    ).agg(
        TOTAL_ATRASO=(f'FLAG_ATRASO_{flag_partida_chegada}', 'count')
    ).reset_index().sort_values(by=[f'MES_{flag_partida_chegada}_PREVISTA'])
    dataframe['TOTAL_ATRASO_DESLOCADO'] = dataframe.groupby(f'FLAG_ATRASO_{flag_partida_chegada}')['TOTAL_ATRASO'].shift(1)
    dataframe.fillna(0, axis=1, inplace=True)
    dataframe[
        [f'FLAG_ATRASO_{flag_partida_chegada}', f'NOME_MES_{flag_partida_chegada}_PREVISTA']
    ] = dataframe[
        [f'FLAG_ATRASO_{flag_partida_chegada}', f'NOME_MES_{flag_partida_chegada}_PREVISTA']
    ].astype('string')
    dataframe[
        [f'MES_{flag_partida_chegada}_PREVISTA', 'TOTAL_ATRASO', 'TOTAL_ATRASO_DESLOCADO']
    ] = dataframe[
        [f'MES_{flag_partida_chegada}_PREVISTA', 'TOTAL_ATRASO', 'TOTAL_ATRASO_DESLOCADO']
    ].astype('int32')
    return dataframe



In [302]:
dataframe = obter_variacao_flag_atraso(
    flag_partida_chegada='PARTIDA',
    flag_atraso='Vôo Atrasado',
    sigla_aeroporto='SBRP'
)
dataframe

,FLAG_ATRASO_PARTIDA,NOME_MES_PARTIDA_PREVISTA,MES_PARTIDA_PREVISTA,TOTAL_ATRASO,TOTAL_ATRASO_DESLOCADO
4,Vôo Atrasado,janeiro,1,227,0
3,Vôo Atrasado,fevereiro,2,201,227
8,Vôo Atrasado,março,3,229,201
0,Vôo Atrasado,abril,4,275,229
7,Vôo Atrasado,maio,5,307,275
6,Vôo Atrasado,junho,6,251,307
5,Vôo Atrasado,julho,7,387,251
1,Vôo Atrasado,agosto,8,252,387
11,Vôo Atrasado,setembro,9,235,252
10,Vôo Atrasado,outubro,10,223,235


In [303]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=dataframe['NOME_MES_PARTIDA_PREVISTA'],
        y=dataframe['TOTAL_ATRASO'],
        name='Mês atual',

    )
)

fig.add_trace(
    go.Scatter(
        x=dataframe['NOME_MES_PARTIDA_PREVISTA'],
        y=dataframe['TOTAL_ATRASO_DESLOCADO'],
        name='Mês anterior'
    )
)

fig.show()

- Analise a FAIXA_ATRASO_CHEGADA / PARTIDA por aeroporto e empresa por dia de semana e por mês

In [369]:
def obter_faixa_atraso_mes_anterior(
        flag_origem_destino: str, 
        flag_partida_chegada: str,
        codigo_tipo_linha: str,
        faixa_atraso: str,
        sigla_aeroporto: str,
        sigla_empresa: str = None,
        
    ):
    colunas = [
        f'NOME_AEROPORTO_{flag_origem_destino}',
        'SIGLA_ICAO_EMPRESA_AEREA',
        f'SIGLA_ICAO_AEROPORTO_{flag_origem_destino}',
        'CODIGO_TIPO_LINHA',
        'NOME_EMPRESA',
        f'FAIXA_ATRASO_{flag_partida_chegada}',
        'SITUACAO_VOO',
        f'NOME_MES_{flag_partida_chegada}_PREVISTA',
        f'MES_{flag_partida_chegada}_PREVISTA'
    ]
    query = (
        f'  CODIGO_TIPO_LINHA == "{codigo_tipo_linha}" ' 
        f' and FAIXA_ATRASO_{flag_partida_chegada} == "{faixa_atraso}" '
        f' and SIGLA_ICAO_AEROPORTO_{flag_origem_destino} == "{sigla_aeroporto}" '
    )
    if sigla_empresa is not None:
        query += f' and SIGLA_ICAO_EMPRESA_AEREA == "{sigla_empresa}" '  
    dataframe = pd.read_parquet(caminho_base, columns=colunas)
    dataframe = dataframe.query(' SITUACAO_VOO != "CANCELADO"')
    dataframe.drop('SITUACAO_VOO', axis=1)
    dataframe = dataframe.query(query)
    dataframe[
        [f'NOME_MES_{flag_partida_chegada}_PREVISTA', f'FAIXA_ATRASO_{flag_partida_chegada}']
    ] = dataframe[
        [f'NOME_MES_{flag_partida_chegada}_PREVISTA',f'FAIXA_ATRASO_{flag_partida_chegada}']
    ].astype('string')
    dataframe = dataframe.groupby(
        [
            f'FAIXA_ATRASO_{flag_partida_chegada}', 
            f'NOME_MES_{flag_partida_chegada}_PREVISTA',
            f'MES_{flag_partida_chegada}_PREVISTA'
        ], observed=False).agg(
        TOTAL_FAIXA=(f'FAIXA_ATRASO_{flag_partida_chegada}', 'count')
    ).reset_index().sort_values(by=[f'FAIXA_ATRASO_{flag_partida_chegada}', f'MES_{flag_partida_chegada}_PREVISTA'])
    dataframe['TOTAL_FAIXA_MES_ANTERIOR'] = dataframe['TOTAL_FAIXA'].shift(1)
    dataframe.fillna('0', axis=1, inplace=True)
    return dataframe


In [370]:
dataframe = obter_faixa_atraso_mes_anterior(
    flag_partida_chegada='CHEGADA',
    sigla_aeroporto="SBRP",
    flag_origem_destino='DESTINO',
    codigo_tipo_linha='N',
    faixa_atraso='Atraso de até 30 minutos'
)
dataframe

,FAIXA_ATRASO_CHEGADA,NOME_MES_CHEGADA_PREVISTA,MES_CHEGADA_PREVISTA,TOTAL_FAIXA,TOTAL_FAIXA_MES_ANTERIOR
4,Atraso de até 30 minutos,janeiro,1,69,0
3,Atraso de até 30 minutos,fevereiro,2,36,69.0
8,Atraso de até 30 minutos,março,3,76,36.0
0,Atraso de até 30 minutos,abril,4,75,76.0
7,Atraso de até 30 minutos,maio,5,105,75.0
6,Atraso de até 30 minutos,junho,6,77,105.0
5,Atraso de até 30 minutos,julho,7,43,77.0
1,Atraso de até 30 minutos,agosto,8,98,43.0
11,Atraso de até 30 minutos,setembro,9,93,98.0
10,Atraso de até 30 minutos,outubro,10,92,93.0


In [354]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=dataframe['NOME_MES_CHEGADA_PREVISTA'],
        y=dataframe['TOTAL_FAIXA'],
        name='Mês atual',

    )
)

fig.add_trace(
    go.Scatter(
        x=dataframe['NOME_MES_CHEGADA_PREVISTA'],
        y=dataframe['TOTAL_FAIXA_MES_ANTERIOR'],
        name='Mês anterior'
    )
)

fig.show()

<p> Ranking por FAIXA_ATRASO_CHEGADA / FAIXA_ATRASO_SAIDA selecionando aeroporto  por mês e empresa</p>
<p>Dentro da faixa Atraso qual compania teve maior incidencia

In [361]:
def obter_top_dez_faixa_atraso(
        flag_origem_destino: str, 
        flag_partida_chegada: str,
        codigo_tipo_linha: str,
        mes: int
    ):
    colunas = [
        f'NOME_AEROPORTO_{flag_origem_destino}',
        'SIGLA_ICAO_EMPRESA_AEREA',
        f'SIGLA_ICAO_AEROPORTO_{flag_origem_destino}',
        'CODIGO_TIPO_LINHA',
        'NOME_EMPRESA',
        f'FAIXA_ATRASO_{flag_partida_chegada}',
        'SITUACAO_VOO',
        f'NOME_MES_{flag_partida_chegada}_PREVISTA',
        f'MES_{flag_partida_chegada}_PREVISTA'
    ]
    query = (
        f'  CODIGO_TIPO_LINHA == "{codigo_tipo_linha}" ' 
        f' and FAIXA_ATRASO_{flag_partida_chegada} == "Atraso de até 30 minutos" '
        f' and MES_{flag_partida_chegada}_PREVISTA == {mes} '
    ) 
    dataframe = pd.read_parquet(caminho_base, columns=colunas)
    return dataframe

In [360]:
dataframe = obter_top_dez_faixa_atraso(
    flag_partida_chegada='CHEGADA',
    flag_origem_destino='DESTINO',
    codigo_tipo_linha='N'
)
dataframe

,NOME_AEROPORTO_DESTINO,SIGLA_ICAO_EMPRESA_AEREA,SIGLA_ICAO_AEROPORTO_DESTINO,CODIGO_TIPO_LINHA,NOME_EMPRESA,FAIXA_ATRASO_CHEGADA,SITUACAO_VOO,NOME_MES_CHEGADA_PREVISTA,MES_CHEGADA_PREVISTA
0,PORTO SEGURO,AZU,SBPS,N,AZUL,No horário,REALIZADO,janeiro,1
1,PORTO SEGURO,AZU,SBPS,N,AZUL,No horário,REALIZADO,janeiro,1
2,PORTO SEGURO,AZU,SBPS,N,AZUL,No horário,REALIZADO,janeiro,1
3,PORTO SEGURO,AZU,SBPS,N,AZUL,No horário,REALIZADO,janeiro,1
4,PORTO SEGURO,AZU,SBPS,N,AZUL,No horário,REALIZADO,fevereiro,2
...,...,...,...,...,...,...,...,...,...
717352,BARRA GRANDE,ABJ,SIRI,N,ABAETE,Atraso de mais de 4 horas,REALIZADO,janeiro,1
717353,BARRA GRANDE,ABJ,SIRI,N,ABAETE,No horário,REALIZADO,dezembro,12
717354,BARRA GRANDE,ABJ,SIRI,N,ABAETE,Atraso de mais de 4 horas,REALIZADO,janeiro,1
717355,BARRA GRANDE,ABJ,SIRI,N,ABAETE,No horário,REALIZADO,dezembro,12
